In [3]:
import os
import sys
os.chdir("../")#let's always place ourself in the main folder

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #the artifacts_root is the key of the dictionary created
                                                # in the yaml file and we can read this key like that instead of
                                                # ["artifacts_root"] because we used the ConfigBox in the common.py file


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion #data ingestion is the other key value of the dictionary in the config.yaml file

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir= config.unzip_dir
        )                                     

        return data_ingestion_config
    



In [9]:
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        file_path: str
        Download, if it doesn't already exists, the zip file with data, don't need a return, just to save the Data
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )

            logger.info(f"{filename} is downloading with the following info: \n{headers}")
        else:
            logger.info(f"{filename} already downloaded and has {get_size(Path(self.config.local_data_file))} size")
        
        return None

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the downloaded zip file, don't need a return, just to save the Data
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        return None



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-08 21:26:08,255: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-08 21:26:08,256: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-08 21:26:08,257: INFO: common: created directory at: artifacts]
[2024-06-08 21:26:08,258: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-08 21:26:10,063: INFO: 2806263458: artifacts/data_ingestion/data.zip is downloading with the following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0d7c42653a864a9d51a41084edb26066cc832d03140a162dc8c18f274f45d69a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7B7C:2A5B9F:6EFBCE:760F70:6664B04F
Accept-Ranges: bytes
Date: Sat, 08 Jun 2024 19:26:09 GMT
Via: 1.1 varnish
X-Served-By: cache-par-lfpg19600